<a href="https://colab.research.google.com/github/YoheiFukuhara/nlp100-2020/blob/main/08.%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88/72_%E6%90%8D%E5%A4%B1%E3%81%A8%E5%8B%BE%E9%85%8D%E3%81%AE%E8%A8%88%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

学習データの事例$x_1$と事例集合$x_1, x_2, x_3, x_4$に対して，クロスエントロピー損失と，行列$W$に対する勾配を計算せよ．なお，ある事例$x_i$に対して損失は次式で計算される．

$$
l_i = - \log [\mbox{事例}x_i\mbox{が}y_i\mbox{に分類される確率}]
$$
ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．

In [1]:
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python --version
!pip show google tensorflow

Python 3.7.12
Name: google
Version: 2.0.3
Summary: Python bindings to the Google search engine.
Home-page: http://breakingcode.wordpress.com/
Author: Mario Vilas
Author-email: mvilas@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: beautifulsoup4
Required-by: 
---
Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: astunparse, gast, google-pasta, wheel, h5py, keras-preprocessing, absl-py, flatbuffers, wrapt, termcolor, numpy, typing-extensions, keras, clang, opt-einsum, grpcio, protobuf, tensorboard, six, tensorflow-estimator
Required-by: kapre


In [3]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks/'
train_raw = tf.data.TFRecordDataset(BASE_PATH+'train.tfrecord')
print(train_raw)

<TFRecordDatasetV2 shapes: (), types: tf.string>


In [4]:
def _parse_function(example_proto):
    # 特徴の記述
    feature_description = {
        'title': tf.io.FixedLenFeature([], tf.string),
        'category': tf.io.FixedLenFeature([], tf.string)}
  
  # 上記の記述を使って入力の tf.Example を処理
    features = tf.io.parse_single_example(example_proto, feature_description)
    X = tf.io.decode_raw(features['title'], tf.float32)
    y = tf.io.decode_raw(features['category'], tf.int32)
    return X, y

In [5]:
dense_layer = tf.keras.layers.Dense(4, activation='softmax', input_dim=300, kernel_initializer='random_uniform')

def print_loss_gradients(batch_size):
    for X, y_true in train_raw.map(_parse_function).batch(batch_size).take(1):

        print('---y_true---')
        print(y_true.numpy())

        with tf.GradientTape() as tape:
            y_pred = dense_layer(X)
            print('---y_pred---')
            print(y_pred.numpy())
            loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
            print('---loss---')
            print(tf.math.reduce_sum((loss.numpy()/batch_size)).numpy())

        grad = tape.gradient(loss, dense_layer.trainable_variables)
        for var, g in zip(dense_layer.trainable_variables, grad):
            print(f'{var.name}, shape: {g.shape}')
            print(g.numpy())

In [6]:
print_loss_gradients(1)

---y_true---
[[1 0 0 0]]
---y_pred---
[[0.24977544 0.23702973 0.2536065  0.25958833]]
---loss---
1.3871931
dense/kernel:0, shape: (300, 4)
[[-0.0627641   0.01983001  0.02121682  0.02171727]
 [-0.00422678  0.00133543  0.00142882  0.00146252]
 [-0.00511264  0.00161531  0.00172828  0.00176905]
 ...
 [-0.05636054  0.01780683  0.01905216  0.01950154]
 [-0.03248982  0.010265    0.01098288  0.01124194]
 [ 0.06511701 -0.02057339 -0.0220122  -0.02253141]]
dense/bias:0, shape: (4,)
[-0.7502246   0.23702973  0.2536065   0.25958833]


In [7]:
print_loss_gradients(4)

---y_true---
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]
---y_pred---
[[0.24977544 0.23702973 0.25360653 0.25958833]
 [0.2663253  0.23270282 0.24324742 0.2577245 ]
 [0.27293497 0.24227263 0.24034856 0.24444385]
 [0.25339198 0.24738249 0.25204796 0.24717763]]
---loss---
1.408023
dense/kernel:0, shape: (300, 4)
[[-0.05385786 -0.00080359  0.08323769 -0.02857625]
 [ 0.01108076 -0.01061149 -0.03302402  0.03255475]
 [-0.035903    0.07073843 -0.08642166  0.05158622]
 ...
 [-0.09915905  0.08452739 -0.00631778  0.02094944]
 [-0.00209951  0.08721732 -0.00053291 -0.08458491]
 [ 0.08135682  0.00262913  0.01224182 -0.09622776]]
dense/bias:0, shape: (4,)
[ 0.04242766 -0.04061237 -0.01074964  0.00893429]
